In [1]:
!wget -q https://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz

In [2]:
!apt-get install openjdk-11-jdk-headless -qq> /dev/null

In [3]:
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [4]:
!pip install -q findspark

In [5]:
!pip install koalas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.8/390.8 KB 7.7 MB/s eta 0:00:00


In [6]:
!pip install --no-deps spark-df-profiling-new

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark-df-profiling-new: filename=spark_df_profiling_new-1.1.14-py3-none-any.whl size=92076 sha256=339502614585355a8ae227440abe8c3ff65fa7f9ba5e78eaada70f211701a51f
  Stored in directory: /root/.cache/pip/wheels/ff/bf/de/f525f8bee6fc166a4216204bcf64477c8f152a380dfa8b5cd0
Successfully built spark-df-profiling-new


In [7]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os

In [8]:
# set spark folder to the system path env
import os
os.environ["JAVA_home"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [9]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
import databricks.koalas as ks
# set koalas
ks.set_option('compute.ops_on_diff_frames', True)

### Read dataset from Google Drive

In [11]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [12]:
mkdir data

In [13]:
cp /content/gdrive/MyDrive/'Colab Notebooks'/dataBDF2/data.zip /content/data/data.zip

In [14]:
!unzip data/data.zip -d data

Archive:  data/data.zip
  inflating: data/credits.csv        
  inflating: data/keywords.csv       
  inflating: data/links_small.csv    
  inflating: data/movies_metadata.csv  
  inflating: data/ratings_small.csv  


In [15]:
from IPython.utils import encoding
from IPython.utils.py3compat import encode
# read files
df_credits = spark.read.options(quote="\"", escape="\"",multiline=True).csv('data/credits.csv', inferSchema=True, header=True).distinct()
df_keywords = spark.read.options(quote="\"", escape="\"",multiline=True).csv('data/keywords.csv', inferSchema=True, header=True).distinct()
df_links = spark.read.options(quote="\"", escape="\"",multiline=True).csv('data/links_small.csv', inferSchema=True, header=True).distinct()
df_movies_metadata = spark.read.options(quote="\"", escape="\"",multiline=True).csv('data/movies_metadata.csv', inferSchema=True, header=True).distinct()
df_ratings_small = spark.read.options(quote="\"", escape="\"",multiline=True).csv('data/ratings_small.csv', inferSchema=True, header=True).distinct()

In [16]:
df_movies_metadata = df_movies_metadata.na.drop(subset=["vote_average"]) #Remove a few wrong data rows

### Raw Data Exploration

Duplicates have been cleared before exploring the data with spark_df_profiling package. 


In [17]:
import spark_df_profiling

In [18]:
mkdir profiles

In [20]:
# Datasets profiling for quick overview in HTML format
# Commented to avoid creating again
"""
profile = spark_df_profiling.ProfileReport(df_credits)
profile.to_file(outputfile="/content/profiles/credits_profile.html")"""

'\nprofile = spark_df_profiling.ProfileReport(df_credits)\nprofile.to_file(outputfile="/content/profiles/credits_profile.html")'

In [21]:
"""profile = spark_df_profiling.ProfileReport(df_keywords)
profile.to_file(outputfile="/content/profiles/keywords_profile.html")"""

'profile = spark_df_profiling.ProfileReport(df_keywords)\nprofile.to_file(outputfile="/content/profiles/keywords_profile.html")'

In [22]:
"""profile = spark_df_profiling.ProfileReport(df_links)
profile.to_file(outputfile="/content/profiles/links_profile.html")"""

'profile = spark_df_profiling.ProfileReport(df_links)\nprofile.to_file(outputfile="/content/profiles/links_profile.html")'

In [23]:
"""profile = spark_df_profiling.ProfileReport(df_movies_metadata)
profile.to_file(outputfile="/content/profiles/movies_metadata_profile.html")"""

'profile = spark_df_profiling.ProfileReport(df_movies_metadata)\nprofile.to_file(outputfile="/content/profiles/movies_metadata_profile.html")'

In [24]:
"""profile = spark_df_profiling.ProfileReport(df_ratings_small)
profile.to_file(outputfile="/content/profiles/ratings_profile.html")"""

'profile = spark_df_profiling.ProfileReport(df_ratings_small)\nprofile.to_file(outputfile="/content/profiles/ratings_profile.html")'

In [25]:
movies = df_movies_metadata.to_koalas()

Some informations didn't properly show up on the profiles such as "budget" column being wrongly displayed as a categorical feature.

In [62]:
movies.info()

<class 'databricks.koalas.frame.DataFrame'>
Int64Index: 45443 entries, 0 to 45442
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45443 non-null  object 
 1   belongs_to_collection  4489 non-null   object 
 2   budget                 45443 non-null  object 
 3   genres                 45443 non-null  object 
 4   homepage               7776 non-null   object 
 5   id                     45443 non-null  object 
 6   imdb_id                45426 non-null  object 
 7   original_language      45432 non-null  object 
 8   original_title         45443 non-null  object 
 9   overview               44489 non-null  object 
 10  popularity             45443 non-null  object 
 11  poster_path            45060 non-null  object 
 12  production_companies   45443 non-null  object 
 13  production_countries   45443 non-null  object 
 14  release_date           45359 non-null  object 
 

Dtype "object" columns may contain multiple types. Otherwise, the type can just be changed to the correct one.

Example with "budget" column:

In [68]:
# Seeking for budget over 300M$ to check max budget value
movies[movies['budget']>300000000]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
18924,False,"{'id': 295, 'name': 'Pirates of the Caribbean ...",380000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",http://disney.go.com/pirates/index-on-stranger...,1865,tt1298650,en,Pirates of the Caribbean: On Stranger Tides,Captain Jack Sparrow crosses paths with a woma...,27.88772,/wNUDAq5OUMOtxMlz64YaCp7gZma.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}, {'...","[{'iso_3166_1': 'US', 'name': 'United States o...",2011-05-14,1045713802,136.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Live Forever Or Die Trying.,Pirates of the Caribbean: On Stranger Tides,False,6.4,5068


In [66]:
# numerical features info :
df_movies_metadata.withColumn("budget", df_movies_metadata.budget.cast('int')).to_koalas().describe()

,budget,revenue,runtime,vote_average,vote_count
count,4.544300e+04,4.544300e+04,45186.000000,45443.000000,45443.000000
mean,4.226417e+06,1.121350e+07,94.126234,5.618311,109.922210
std,1.742775e+07,6.434392e+07,38.412464,1.924092,491.398368
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,0.000000e+00,107.000000,6.800000,34.000000
max,3.800000e+08,2.787965e+09,1256.000000,10.000000,14075.000000
